## ライブラリの準備

In [ ]:
'''GoogleColaboratoryで実行する場合は以下のコマンドを実行
'''
# !pip install transformers==4.21.2 fugashi==1.1.2 ipadic==1.0.0

In [ ]:
# データ加工に用いるライブラリ
import json
import unicodedata
import itertools
import re

# 出力整理に用いるライブラリ
import pandas as pd
from tqdm import tqdm
import random

# torch, BERT
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification

## データセットの準備

In [ ]:
# データのダウンロード
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset

# データのロード
dataset = json.load(open('ner-wikipedia-dataset/ner.json', 'r'))

In [ ]:
# 固有表現タイプの辞書
type_id_dictionary = {'人名': 1,
                      '法人名': 2,
                      '政治的組織名': 3,
                      'その他の組織名': 4,
                      '地名': 5,
                      '施設名': 6,
                      '製品名': 7,
                      'イベント名': 8}

'''idからtypeを取得する関数
'''
def get_type_from_id(id):
    keys = [key for key, value in type_id_dictionary.items() if value == id]
    if keys:
        return keys[0]
    return None

In [ ]:
# 前処理
for data in dataset:
    # 正規化
    # ｱｲｳ → アイウ, ＡＢＣ → ABC, １２３ → 123
    data['text'] = unicodedata.normalize('NFKC', data['text'])

    # typeを対応するtype_idに変換
    for entity in data['entities']:
        entity['type_id'] = type_id_dictionary[entity['type']]
        del entity['type']

In [ ]:
# データセットの分割
# 学習：検証：テスト = 6：2：2
size = len(dataset)
size_train_dataset = int(0.6 * size)
size_val_dataset = int(0.2 * size)

train_dataset = dataset[: size_train_dataset]
val_dataset = dataset[size_train_dataset : (size_train_dataset + size_val_dataset)]
test_dataset = dataset[size_train_dataset + size_val_dataset :]

# 学習・検証・テストデータのサイズを表示
print(f'学習データ数   : {len(train_dataset)}')
print(f'検証データ数   : {len(val_dataset)}')
print(f'テストデータ数 : {len(test_dataset)}')

## トークナイザの準備

In [ ]:
'''固有表現抽出に適応したBertJapaneseTokenizerを拡張したトークナイザ
'''
class ExtensionTokenizer(BertJapaneseTokenizer):

    # 学習時に用いるラベル付きエンコーダ
    def tagged_encode_plus(self, data, max_length):
        text = data['text']         # dataから文字列を取得
        entities = data['entities'] # dataから固有表現を取得(出現順でソート)

        '''[Step1] 固有表現かそれ以外かで分割
        '''
        entities = sorted(entities, key=lambda x: x['span'][0]) # 固有表現の位置の昇順でソート

        data_splitted = [] # 分割後の文字列格納用
        head = 0           # 文字列の先頭のindex

        for entity in entities:
            # 次に出現する固有表現の先頭・末尾・IDを取得
            entity_head = entity['span'][0]
            entity_tail = entity['span'][1]
            label = entity['type_id']

            # 固有表現にID、固有表現以外に'0'をラベルとして付与
            data_splitted.append({'text': text[head:entity_head], 'label':0})
            data_splitted.append({'text': text[entity_head:entity_tail], 'label':label})

            head = entity_tail  # 先頭indexを更新

        # 最後の固有表現以降のtextに'0'をラベルとしてを付与
        data_splitted.append({'text': text[head:], 'label':0})

        # head = entity_startの時、{'text': '', 'label': 0}となってしまうため、textが空の要素を削除
        data_splitted = [ s for s in data_splitted if s['text'] ]

        '''[Step2] トークナイザを用い、分割された文字列をトークン化・ラベル付与
        '''
        tokens = []
        labels = []

        for s in data_splitted:
            text_splitted = s['text']
            label_splitted = s['label']

            tokens_splitted = self.tokenize(text_splitted)        # トークン化
            labels_splitted = [label_splitted] * len(tokens_splitted)  # 各トークンにラベル付与

            tokens.extend(tokens_splitted)  # トークンを結合
            labels.extend(labels_splitted)  # ラベルを結合

        '''[Step3] BERTに入力可能な形式に符号化
        '''
        encoding = self.encode_plus(tokens,
                                    max_length=max_length,
                                    padding='max_length',
                                    truncation=True,
                                    return_tensors='pt')

        # トークン[CLS]、[SEP]に'0'ラベルとして付与
        labels = [0] + labels[:max_length-2] + [0]
        # トークン[PAD]に'0'をラベルとして付与
        labels = labels + [0]*( max_length - len(labels) )

        encoding['labels'] = torch.tensor([labels])
        return encoding

    # テスト時に用いるencordingとspansを返すエンコーダ
    def untagged_encode_plus(self, text, max_length):
        '''[Step1] BERTに入力可能な形式に符号化
        '''
        encoding = self.encode_plus(text=text,
                                    max_length=max_length,
                                    padding='max_length',
                                    truncation=True,
                                    return_tensors = 'pt')

        '''[Step2]各トークンのスパンを格納
        '''
        spans = []

        tokens = self.convert_ids_to_tokens(encoding['input_ids'][0])
        head = 0

        for token in tokens:
            # '##'は文字数にカウントしないので読み飛ばす
            token = token.replace('##','')

            # スペシャルトークンの場合はダミーとしてspanを[-1, -1]とする
            if token == '[PAD]':
                spans.append([-1, -1])
            elif token == '[UNK]':
                spans.append([-1, -1])
            elif token == '[CLS]':
                spans.append([-1, -1])
            elif token == '[SEP]':
                spans.append([-1, -1])

            # text中からtokenをを探索し，開始位置 + 文字列長をspanとする
            # トークンが見つかるまでスペースを読み飛ばす
            else:
                length = len(token)
                while 1:
                    if token == text[head:head+length]:
                        spans.append([head, head+length])
                        head += length
                        break

                    head += 1

        return encoding, spans

In [ ]:
# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# トークナイザをロード
tokenizer = ExtensionTokenizer.from_pretrained(MODEL_NAME)

## データセット・データローダの作成

In [ ]:
'''データローダに格納するデータセット
'''
class ExtensionDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        encoding = self.tokenizer.tagged_encode_plus(self.dataset[index], self.max_length)
        return encoding

In [ ]:
# データセットの作成
dataset_train_for_loader = ExtensionDataset(dataset = train_dataset,
                                            tokenizer = tokenizer,
                                            max_length = 128)

dataset_val_for_loader = ExtensionDataset(dataset = val_dataset,
                                          tokenizer = tokenizer,
                                          max_length = 128)

# データローダの作成
dataloader_train = DataLoader(dataset_train_for_loader, batch_size=16, shuffle=True, pin_memory=True)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256, shuffle=True, pin_memory=True)

dataloaders = {'train': dataloader_train, 'val': dataloader_val}

## 事前学習モデル

In [ ]:
# 学習済みモデルをロード
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=9)

print(f'\nmodelのパラメータを確認\n{model.get_parameter}')

### 予測

In [ ]:
'''文字列の符号化、BERTによる推論、BERTの出力をentitiesに変換する関数
'''
def predict(text, tokenizer, max_length, model):
    # 符号化
    encoding, spans = tokenizer.untagged_encode_plus(text, max_length)

    # モデルとデータをGPUまたはCPUに乗せる
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    encoding = { key:value.to(device) for key, value in encoding.items() }

    # 予測
    with torch.no_grad():
        output = model(**encoding)

    # 最も高い確率のクラスを予測ラベルとする
    predicted_label = output.logits[0].argmax(-1).cpu().numpy().tolist()

    # スペシャルトークンを削除
    predicted_label = [label for label, span in zip(predicted_label, spans) if span[0] != -1]
    spans = [span for span in spans if span[0] != -1]

    # 同じラベルが連続するトークンをまとめる
    entities = []
    label_head = 0  # 連続するラベルの先頭
    for label, group in itertools.groupby(predicted_label):
        label_tail = label_head + len(list(group)) - 1 # 連続するラベルの末尾

        # 予測固有表現をentitiesに格納
        head = spans[label_head][0]
        tail = spans[label_tail][1]
        if label != 0:
            entity = {'name': text[head:tail],
                      'span': [head, tail],
                      'type_id': label}

            entities.append(entity)

        label_head = label_tail + 1

    return entities

In [ ]:
entities_list = []            # 正解固有表現
predicted_entities_list = []  # 予測固有表現

for data in tqdm(test_dataset):
    entities_list.append(data['entities'])

    # 固有表現抽出
    predicted_entities = predict(data['text'], tokenizer, 128, model)
    predicted_entities_list.append(predicted_entities)

In [ ]:
# 結果をランダムに確認
for i in range(3):
    index = random.randint(0, len(test_dataset) - 1)

    print(f'text[{index:5}] : {test_dataset[index]["text"]}')
    print(f'正解固有表現 : {entities_list[index]}')
    print(f'予測固有表現 : {predicted_entities_list[index]}\n')

### 性能評価

In [ ]:
'''適合率、再現率、F値を計算し、モデルを評価する関数
'''
def evaluate(dataset, entities_list, predicted_entities_list, type_id=None):
    entities_count = 0            # 正解固有表現の個数
    predicted_entities_count = 0  # 予測固有表現の個数
    correct_count = 0             # 予測固有表現うち正解の個数

    for entities, predicted_entities in zip(entities_list, predicted_entities_list):

        # 引数type_idが指定された場合、そのクラスの固有表現のみを抽出
        if type_id:
            entities = [ entity for entity in entities if entity['type_id'] == type_id ]
            predicted_entities = [ entity for entity in predicted_entities if entity['type_id'] == type_id ]

        # 重複固有表現をset型に変換
        get_span_type = lambda entity: (entity['span'][0], entity['span'][1], entity['type_id'])
        set_entities = set( get_span_type(entity) for entity in entities )
        set_entities_predicted = set( get_span_type(entity) for entity in predicted_entities )

        # 各個数を更新
        entities_count += len(entities)
        predicted_entities_count += len(predicted_entities)
        correct_count += len( set_entities & set_entities_predicted )

    precision = correct_count / predicted_entities_count    # 適合率
    recall = correct_count / entities_count                 # 再現率
    if(precision + recall != 0):
        f_value = 2 * precision*recall / (precision + recall) # F値
    else:
        f_value = -1

    result = {'entities_count': entities_count,
              'predicted_entities_count': predicted_entities_count,
              'correct_count': correct_count,
              'precision': precision,
              'recall': recall,
              'f_value': f_value}

    return result

In [ ]:
evaluation_df = pd.DataFrame()

# 各クラスの予測性能を評価
for key, value in type_id_dictionary.items():
    evaluation = evaluate(test_dataset, entities_list, predicted_entities_list, type_id=value)
    evaluation_df[key] = evaluation.values()  # 各列に評価結果を格納

# 全クラスの予測性能を評価
evaluation_all = evaluate(test_dataset, entities_list, predicted_entities_list, type_id=None)
evaluation_df['ALL'] = evaluation_all.values()  #　全クラスの結果を末尾の列に格納

# 行名を設定
evaluation_df.index = evaluation_all.keys()

evaluation_df

## ファインチューニング

### 学習

In [ ]:
'''モデルをファインチューニングする関数
'''
def train(model, dataloaders, optimizer, max_epoch):

    # モデルをGPUまたはCPUに乗せる
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    print(f'使用デバイス：{device}')

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    train_average_loss_list = []
    val_average_loss_list = []
    history = {}

    # epochのループ
    for epoch in range(max_epoch):
        print(f'\nepoch [{epoch+1}/{max_epoch}]')

        '''[Step1]学習
        '''
        model.train()

        iteration = 1
        sum_loss = 0.0

        # ミニバッチを取り出す
        for batch in tqdm(dataloaders['train']):
            input_ids = batch['input_ids'][0].to(device)
            attention_mask = batch['attention_mask'][0].to(device)
            labels = batch['labels'][0].to(device)

            optimizer.zero_grad() # optimizerを初期化

            loss, logits = model(input_ids = input_ids,
                                 token_type_ids = None,
                                 attention_mask = attention_mask,
                                 labels = labels,
                                 return_dict = False)

            loss.backward() # 順伝搬
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 勾配クリッピング
            optimizer.step()  # 最適化

            # 1エポックの損失の和を更新
            sum_loss += loss.item() * dataloaders['train'].batch_size

            iteration += 1

        # 1エポックの平均損失を記録
        average_loss = sum_loss / len(dataloaders['train'].dataset)
        train_average_loss_list.append(average_loss)

        '''[Step2]検証
        '''
        model.eval()

        iteration = 1
        sum_loss = 0.0
        val_accuracy = 0.0

        # ミニバッチを取り出す
        for batch in (dataloaders['val']):
            input_ids = batch['input_ids'][0].to(device)
            attention_mask = batch['attention_mask'][0].to(device)
            labels = batch['labels'][0].to(device)

            optimizer.zero_grad() # optimizerを初期化

            loss, logits = model(input_ids = input_ids,
                                 token_type_ids = None,
                                 attention_mask = attention_mask,
                                 labels = labels,
                                 return_dict = False)

            # 1エポックの損失の和を更新
            sum_loss += loss.item() * dataloaders['val'].batch_size

            iteration += 1

        # 1エポックの平均損失を記録
        average_loss = sum_loss / len(dataloaders['val'].dataset)
        val_average_loss_list.append(average_loss)

        print(f'train_loss: {train_average_loss_list[epoch]:.4f}, val_loss: {val_average_loss_list[epoch]:.4f}')

    history['train_loss'] = train_average_loss_list
    history['val_loss'] = val_average_loss_list

    return model,history

In [ ]:
max_epoch = 5   # エポック数
lr = 2e-5       # 学習率

# 最適化器としてAdamを使用
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

# ファインチューニング
trained_model, history = train(model = model,
                               dataloaders = dataloaders,
                               optimizer = optimizer,
                               max_epoch=max_epoch)

In [ ]:
# 学習曲線の表示
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.plot(history['train_loss'],label='train', c='b')
plt.plot(history['val_loss'],label='val', c='r')
plt.title('learning curve')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

### 予測

In [ ]:
trained_entities_list = []           # 正解固有表現
trained_predicted_entities_list = [] # 予測固有表現

for data in tqdm(test_dataset):
    trained_entities_list.append(data['entities'])

    # 固有表現抽出
    predicted_entities = predict(data['text'], tokenizer, 128, trained_model)
    trained_predicted_entities_list.append(predicted_entities)

In [ ]:
# 結果をランダムに確認
for i in range(3):
    index = random.randint(0, len(test_dataset) - 1)

    print(f'text[{index:5}] : {test_dataset[index]["text"]}')
    print(f'正解固有表現 : {trained_entities_list[index]}')
    print(f'予測固有表現 : {trained_predicted_entities_list[index]}\n')

### 性能評価

In [ ]:
evaluation_df = pd.DataFrame()

# 各クラスの予測性能を評価
for key, value in type_id_dictionary.items():
    evaluation = evaluate(test_dataset, trained_entities_list, trained_predicted_entities_list, type_id=value)
    evaluation_df[key] = evaluation.values()  # 各列に評価結果を格納

# 全クラスの予測性能を評価
evaluation_all = evaluate(test_dataset, trained_entities_list, trained_predicted_entities_list, type_id=None)
evaluation_df['ALL'] = evaluation_all.values()  # 全クラスの結果を末尾の列に格納

# 行名を設定
evaluation_df.index = evaluation_all.keys()

evaluation_df

## 銀河鉄道の夜の固有表現抽出

In [ ]:
!wget https://www.aozora.gr.jp/cards/000081/files/456_ruby_145.zip
!unzip 456_ruby_145.zip

In [ ]:
with open('gingatetsudono_yoru.txt', mode='rb') as f:
    text = f.read().decode('shift_jis')

print(text)

In [ ]:
#前処理
# ヘッダとフッタの削除
text = re.split(r'\-{5,}',text)[2]
text = re.split(r'底本：', text)[0]
text = text.strip() # 連続する改行文字の削除

text = re.sub(r'《.+?》', '', text)     # ルビを削除
text =text.replace('｜', '')            # ルビの付を削除
text = re.sub(r'［＃.+?］', '', text)   # 入力者注を削除

text = unicodedata.normalize('NFKC', text)

print(text)

In [ ]:
novel_predicted_entities_list = [] # 予測固有表現

# テキストを1行ずつ取り出し
lines = text.split("\n")
for line in tqdm(lines):
    predicted_entities = predict(line, tokenizer, 128, trained_model)
    novel_predicted_entities_list.extend(predicted_entities)

In [ ]:
# idごとに振り分け
novel_predicted_entities_map = {'人名': [],
                                '法人名': [],
                                '政治的組織名': [],
                                'その他の組織名': [],
                                '地名': [],
                                '施設名':[],
                                '製品名':[],
                                'イベント名':[]}

for entity in novel_predicted_entities_list:
    if entity['type_id'] == 1:
        novel_predicted_entities_map['人名'].append(entity['name'])
    elif entity['type_id'] == 2:
        novel_predicted_entities_map['法人名'].append(entity['name'])
    elif entity['type_id'] == 3:
        novel_predicted_entities_map['政治的組織名'].append(entity['name'])
    elif entity['type_id'] == 4:
        novel_predicted_entities_map['その他の組織名'].append(entity['name'])
    elif entity['type_id'] == 5:
        novel_predicted_entities_map['地名'].append(entity['name'])
    elif entity['type_id'] == 6:
        novel_predicted_entities_map['施設名'].append(entity['name'])
    elif entity['type_id'] == 7:
        novel_predicted_entities_map['製品名'].append(entity['name'])
    elif entity['type_id'] == 8:
        novel_predicted_entities_map['イベント名'].append(entity['name'])

In [ ]:
# 重複要素削除して抽出結果を表示
for key, value in type_id_dictionary.items():
    novel_predicted_entities_map[key] = set(novel_predicted_entities_map[key])
    print(f'\n[{key}]')
    for entity in novel_predicted_entities_map[key]:
        print(entity)